# Compiling a QGL2 HahnEcho and plotting the output

## Imports

In [ ]:
from pyqgl2.main import compile_function, qgl2_compile_to_hardware
from pyqgl2.test_cl import create_default_channelLibrary
from pyqgl2.qreg import QRegister
from QGL import plot_pulse_files, ChannelLibrary
import numpy as np

### Should sequences be compiled to hardware, or just to QGL?

In [ ]:
toHW = True

### Create a test ChannelLibrary

In [ ]:
create_default_channelLibrary(toHW, True)
# Alternatively could load an existing library, or create one here; see the 'AllXY' notebook

## Create needed qubit(s)

In [ ]:
# For QGL2, use a QRegister, not a QGL Qubit
q = QRegister(1)

## Compile to QGL1

### To turn on debug output, uncomment the next 4 lines

In [ ]:
#from pyqgl2.ast_util import NodeError
#from pyqgl2.debugmsg import DebugMsg

#DebugMsg.set_level(1)
#NodeError.MUTE_ERR_LEVEL = NodeError.NODE_ERROR_NONE

In [ ]:
# Insert proper path to QGL2 source and name of qgl2main if not so marked
# Here we compile the named function in the named file from QGL2 to QGL1 and return the new function

In [ ]:
tCalR = 2 # calRepeats
hahnSpacings = np.linspace(0, 5e-6, 11)
qgl1MainFunc = compile_function("../src/python/qgl2/basic_sequences/Decoupling.py", "HahnEcho", 
                                (q, hahnSpacings, 0, tCalR))

## Generate pulse sequences

In [ ]:
# Now run the QGL1 function, producing a list of sequences
seqs = qgl1MainFunc()

## Optionally compile to machine instructions

In [ ]:
if toHW:
    from qgl2.basic_sequences.helpers import delay_descriptor, cal_descriptor
    axis_desc = [
            delay_descriptor(2 * hahnSpacings),
            cal_descriptor(('qubit',), tCalR)
        ]
    metaFileName = qgl2_compile_to_hardware(seqs, "Echo/Echo", axis_descriptor=axis_desc)
    print(f"Generated sequence details in '{metaFileName}'")
    # Plot the sequences
    p = plot_pulse_files(metaFileName)
    # Explicitly display the graph which fails to auto-draw in some cases
    display(p)
else:
    from QGL.Scheduler import schedule
    from IPython.lib.pretty import pretty
    print(pretty(schedule(seqs)))